In [1]:
import MetaTrader5 as mt
from datetime import datetime

In [2]:
mt.initialize()

True

In [3]:
import json

# Load JSON config file
with open("config.json", "r") as file:
    config = json.load(file)

In [4]:
mt.login(config["account_info"]["login"],config["account_info"]["password"],config["account_info"]["server"])

True

In [5]:
rates = mt.copy_rates_from('EURUSD', mt.TIMEFRAME_M15, datetime.now(),10000)
rates

array([(1722247200, 1.0858 , 1.08598, 1.08523, 1.08523, 710, 1, 0),
       (1722248100, 1.08523, 1.08525, 1.08475, 1.08509, 891, 1, 0),
       (1722249000, 1.08509, 1.08561, 1.08505, 1.0851 , 787, 1, 0), ...,
       (1734700500, 1.03855, 1.03883, 1.03801, 1.03882, 926, 1, 0),
       (1734701400, 1.03881, 1.03974, 1.03872, 1.03907, 918, 1, 0),
       (1734702300, 1.03906, 1.03946, 1.03902, 1.03914, 772, 1, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [6]:
account_info = mt.account_info()
account_info

AccountInfo(login=600011860, trade_mode=0, leverage=500, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=99950.0, credit=0.0, profit=0.0, equity=99950.0, margin=0.0, margin_free=99950.0, margin_level=0.0, margin_so_call=80.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Ashkan Rahdar', server='Opogroup-Server1', currency='USD', company='Opo group LLC')

In [7]:
import pandas as pd

In [8]:
DataSet = pd.DataFrame(rates)
DataSet

,time,open,high,low,close,tick_volume,spread,real_volume
0,1722247200,1.08580,1.08598,1.08523,1.08523,710,1,0
1,1722248100,1.08523,1.08525,1.08475,1.08509,891,1,0
2,1722249000,1.08509,1.08561,1.08505,1.08510,787,1,0
3,1722249900,1.08511,1.08514,1.08432,1.08446,808,1,0
4,1722250800,1.08444,1.08444,1.08381,1.08383,935,1,0
...,...,...,...,...,...,...,...,...
9995,1734698700,1.03888,1.03933,1.03869,1.03919,857,1,0
9996,1734699600,1.03919,1.03919,1.03796,1.03855,785,1,0
9997,1734700500,1.03855,1.03883,1.03801,1.03882,926,1,0
9998,1734701400,1.03881,1.03974,1.03872,1.03907,918,1,0


In [9]:
## for buying price = mt.symbol_info_tick(ticker).ask
## for selling price = mt.symbol_info_tick(ticker).bid

## order_type= mt.ORDER_TYPE_SELL / BUY

def Open_position(ticker, vol, order_type, price, sl, tp):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": vol,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 101,
        "comment": "My Code opened",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK
    }
    order = mt.order_send(request)
    return order

In [10]:
#result = Open_position('BTCUSD',0.01, mt.ORDER_TYPE_SELL, mt.symbol_info_tick('BTCUSD').bid, 68000.0, 66000.0)

#result.retcode

In [11]:
## mt.position_get()[0]._asdict('ticket')
def Close_position(ticker, vol, order_type, price, position):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": vol,
        "type": order_type,
        "position": position,
        "price": price,
        "deviation": 20,
        "magic": 101,
        "comment": "My Code closed",
        "type_time": mt.ORDER_TIME_GTC, 
        "type_filling": mt.ORDER_FILLING_FOK,
    }
    order = mt.order_send(request)
    return order   

In [12]:
#result = Close_position("BTCUSD", 0.01,mt.ORDER_TYPE_BUY ,mt.symbol_info_tick('BTCUSD').ask ,mt.positions_get()[0]._asdict()['ticket'])
#print(result)

In [13]:
import logging
from logging.handlers import RotatingFileHandler
import numpy as np
import typing
from dataclasses import dataclass
import plotly.express as pt
import plotly.graph_objects as go

# Create a custom filter class for log levels
class LevelFilter(logging.Filter):
    def __init__(self, level):
        self.level = level

    def filter(self, record):
        return record.levelno == self.level

# Create two handlers: one for info logs and one for error logs and one for warnnig logs
info_handler = logging.FileHandler('info.log')
info_handler.setLevel(logging.INFO)

error_handler = logging.FileHandler('error.log')
error_handler.setLevel(logging.ERROR)

Warning_handler = logging.FileHandler('Warning.log')
Warning_handler.setLevel(logging.WARNING)

# Add filters to the handlers
info_handler.addFilter(LevelFilter(logging.INFO))
error_handler.addFilter(LevelFilter(logging.ERROR))
Warning_handler.addFilter(LevelFilter(logging.WARNING))

# Create a formatter and set it for both handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
info_handler.setFormatter(formatter)
error_handler.setFormatter(formatter)
Warning_handler.setFormatter(formatter)

# Create a logger and add both handlers
logger = logging.getLogger('TradingBot')
logger.setLevel(logging.DEBUG)  # Log all levels (INFO, ERROR, DEBUG, etc.)
logger.addHandler(info_handler)
logger.addHandler(error_handler)
logger.addHandler(Warning_handler)

# Example log entries
logger.info("Logger is initialized...")

# Set up rotating file handler for info and error logs
info_handler = RotatingFileHandler('info.log', maxBytes=10**6, backupCount=5)
error_handler = RotatingFileHandler('error.log', maxBytes=10**6, backupCount=5)
Warning_handler = RotatingFileHandler('Warning.log', maxBytes= 10**6, backupCount=5)

In [14]:
@dataclass
class FlagPoint:
    price: int
    index: int

In [15]:
@dataclass
class DP_Parameteres:  
    High: FlagPoint 
    Low: FlagPoint
    Status: typing.Literal["Used", "Active", "Not Valid"] = "Active"
    weight: int = 0
    start_index : int = -1

In [16]:
class Important_DPs:
    def __init__(self, 
                 dataset: pd.DataFrame, 
                 direction: typing.Literal["Bullish", "Bearish","Undefined"], 
                 flag_id: pd.Series,
                 start_of_index: int):

        logger.info(f"Detecting Important DPs of Flag {flag_id}")
        highs = dataset['high'].to_numpy()
        lows = dataset['low'].to_numpy()
        local_Lows_index = np.where(dataset['is_local_min'].to_numpy())[0]
        local_Highs_index = np.where(dataset['is_local_max'].to_numpy())[0]

        self.DP = self.DP_Detector(lows, highs, direction, flag_id, local_Lows_index, local_Highs_index, start_of_index)

        """

        self.Tasvie = DP_Parameteres(FlagPoint(None,None),FlagPoint(None,None))

        if (self.DP.Low.price != None and self.DP.High.price != None):
            if direction == "Bullish":
                end = self.DP.Low.index - 1 - start_of_index
            else:
                end = self.DP.High.index - 1 - start_of_index
            #end = min(len(dataset) - 1, end)
            Tdataset = dataset.iloc[:end]
            self.Tasvie_detector(Tdataset, direction, flag_id, start_of_index)
        """

    def DP_Detector(self, 
                    lows: np.ndarray, 
                    highs: np.ndarray, 
                    flag_type: typing.Literal["Bullish", "Bearish","Undefined"], 
                    flag_id: pd.Series,  
                    local_Lows_index: np.ndarray, 
                    local_Highs_index: np.ndarray,
                    start_of_index: int):
        
        DP = DP_Parameteres(FlagPoint(None,None),FlagPoint(None,None),)
        # Detecting DP of Bullish Flag
        if flag_type == "Bullish":
            if len(local_Lows_index) == 0:
                logger.warning(f"No valid Tasvie for Flag -> {flag_id}")
                DP.Status = "Not Valid"
                return DP
            local_Lows = lows[local_Lows_index]
            low_of_DP = local_Lows.min()
            low_of_DP_index = local_Lows_index[np.where(local_Lows == low_of_DP)[0][-1]]
            DP.Low.index = low_of_DP_index + start_of_index
            DP.Low.price = low_of_DP
            
            # Ensure non-empty slice for highs
            highs_slice = highs[low_of_DP_index+1:]
            if highs_slice.size > 0:
                high_of_DP = highs_slice.max()
                high_of_DP_index = (low_of_DP_index+1) + np.where(highs_slice == high_of_DP)[0][-1]
                DP.High.index = high_of_DP_index + start_of_index
                DP.High.price = high_of_DP
                DP.weight = 1
            else:
                DP.Status = "Not Valid"
                logger.warning(f"No highs found in the range for Flag -> {flag_id}")
        # Detecting DP of Bearish Flag
        elif flag_type == "Bearish":
            if len(local_Highs_index) == 0:
                logger.warning(f"No valid Tasvie for Flag -> {flag_id}")
                DP.Status = "Not Valid"
                return DP
            local_highs = highs[local_Highs_index]
            high_of_DP = local_highs.max()
            high_of_DP_index = local_Highs_index[np.where(local_highs == high_of_DP)[0][-1]]
            DP.High.index = high_of_DP_index + start_of_index
            DP.High.price = high_of_DP

            # Ensure non-empty slice for lows
            lows_slice = lows[high_of_DP_index+1:]
            if lows_slice.size > 0:
                low_of_DP = lows_slice.min()
                low_of_DP_index = (high_of_DP_index+1) + np.where(lows_slice == low_of_DP)[0][-1]
                DP.Low.index = low_of_DP_index + start_of_index
                DP.Low.price = low_of_DP
                DP.weight = 1
            else:
                DP.Status = "Not Valid"
                logger.warning(f"No lows found in the range for Flag -> {flag_id}")
        return DP
    
    def Tasvie_detector(self, 
                        dataset: pd.DataFrame, 
                        flag_type: typing.Literal["Bullish", "Bearish","Undefined"], 
                        flag_id: pd.Series,
                        start_of_index: int):
        
        highs = dataset['high'].to_numpy()
        lows = dataset['low'].to_numpy()
        local_Lows_index = np.where(dataset['is_local_min'].to_numpy())[0]
        local_Highs_index = np.where(dataset['is_local_max'].to_numpy())[0]
        local_Low = lows[local_Lows_index]
        local_High = highs[local_Highs_index]

        # Detecting Tasvie of Bearish Flag
        if flag_type == "Bearish":
            is_valid_highs = (local_High > self.DP.Low.price) & (local_High < self.DP.High.price)
            valid_highs_indexes = np.where(is_valid_highs)[0]

            if valid_highs_indexes.size ==0 :
                return
            valids = local_Highs_index[valid_highs_indexes]

            for i in valids:
                high_of_Tasvie = highs[i]
                low_of_Tasvie = lows[i:].min()
                if low_of_Tasvie > self.DP.Low.price:
                    break
                low_of_Tasvie_index = i + np.where(lows[i:] == low_of_Tasvie)[0][0]

                if i == low_of_Tasvie_index:
                    break
                if highs[i:low_of_Tasvie_index].max() == high_of_Tasvie:
                    self.Tasvie.High.price = high_of_Tasvie
                    self.Tasvie.High.index = i + start_of_index
                    self.Tasvie.Low.price = low_of_Tasvie
                    self.Tasvie.Low.index = low_of_Tasvie_index + start_of_index
                    break
            
        # Detecting Tasvie of Bullish Flag
        else:
            is_valid_lows = (local_Low > self.DP.Low.price) & (local_Low < self.DP.High.price)
            valid_lows_indexes = np.where(is_valid_lows)[0]

            if valid_lows_indexes.size ==0 :
                return
            valids = local_Lows_index[valid_lows_indexes]

            for i in valids:
                low_of_Tasvie = lows[i]
                high_of_Tasvie = highs[i:].max()
                if high_of_Tasvie < self.DP.High.price:
                    break
                high_of_Tasvie_index = i + np.where(highs[i:] == high_of_Tasvie)[0][0]

                if i == high_of_Tasvie_index:
                    break
                if lows[i:high_of_Tasvie_index].min() == low_of_Tasvie:
                    self.Tasvie.Low.price = low_of_Tasvie
                    self.Tasvie.Low.index = i + start_of_index
                    self.Tasvie.High.price = high_of_Tasvie
                    self.Tasvie.High.index = high_of_Tasvie_index + start_of_index
                    break
            

    def __repr__(self):
        #return f"Dp: {self.DP}, Tasvie: {self.Tasvie}"
        return f"Dp: {self.DP}"

In [17]:
class Flag:
    def __init__(self, 
                 flag_id: pd.Series, 
                 flag_type: typing.Literal["Bullish", "Bearish","Undefined"], 
                 high: FlagPoint, 
                 low: FlagPoint, 
                 data_in_flag: pd.DataFrame, 
                 start_index: int, 
                 end_index: int, 
                 start_FTC: int,
                 start_EL: int):
        
        self.flag_id = flag_id
        self.flag_type = flag_type
        self.high = high
        self.low = low
        self.duration = end_index - (low.index if flag_type == "Bearish" else high.index)
        self.Start_index = start_index
        self.End_index = end_index

        self.FTC = Important_DPs(
            dataset= (data_in_flag.iloc[high.index - start_index:low.index - start_index] if flag_type=="Bullish" 
                      else data_in_flag.iloc[low.index - start_index:high.index - start_index]),
            direction = self.flag_type,
            flag_id= self.flag_id,
            start_of_index = high.index if flag_type == "Bullish" else low.index
            )
        self.FTC.DP.start_index = start_FTC
        
        if flag_type == "Bullish":
            EL_direction = "Bearish"
        elif flag_type == "Bearish":
            EL_direction = "Bullish"
        else:
            EL_direction = "Undefined"

        self.EL = Important_DPs(
            dataset= (data_in_flag.iloc[:high.index - start_index] if flag_type=="Bullish" 
                      else data_in_flag.iloc[:low.index - start_index]),
            direction = EL_direction,
            flag_id= self.flag_id,
            start_of_index= self.Start_index
        )

        self.EL.DP.start_index = start_EL
        self.weight = self.weight_of_flag()

    def weight_of_flag(self):
        weights = []
        max_weight  = 5
        # duration of flag

        if(self.duration / 15 < max_weight): weights.append(self.duration / 15)
        else: weights.append(max_weight)
        
        return sum(weights)
    def __repr__(self):
        return (f"Flag is like ID: {self.flag_id}, Type: {self.flag_type}, High: {self.high}, Low: {self.low}, "
                f"Start index: {self.Start_index}, End index: {self.End_index}, EL: {self.EL}, "
                f"Duration: {self.duration}, FTC: {self.FTC}")

In [65]:
class FlagDetector:
    """Detects Bullish and Bearish flags from a dataset."""
    def __init__(self, dataset: pd.DataFrame):
        self.dataset = dataset
        self.flags = {} 

    def detect_local_extremes(self):
        """Precompute local maxima and minima for the dataset."""
        highs = self.dataset['high']
        lows = self.dataset['low']

        # Using NumPy for vectorized operations
        is_local_max = (highs > np.roll(highs, 1)) & (highs > np.roll(highs, -1))
        is_local_min = (lows < np.roll(lows, 1)) & (lows < np.roll(lows, -1))

        # Assign results back to the dataset
        self.dataset['is_local_max'] = is_local_max
        self.dataset['is_local_min'] = is_local_min

    def detect_bullish_flags(self):
        """Detect Bullish Flags."""
        logger.info("Detecting Bullish Flags...")
        highs = self.dataset['high']
        lows = self.dataset['low']

        local_max_indices = np.where(self.dataset['is_local_max'].to_numpy())[0]
        for i in local_max_indices:
            high_of_flag = highs[i]

            # Find end of flag
            #end_of_flag_indices = np.where((highs[i + 1:] > high_of_flag) & (lows[i + 1:] > high_of_flag))[0] if i+1 < len(lows) else np.array([])
            end_of_flag_indices = np.where((highs[i + 1:] > high_of_flag))[0] if i+1 < len(lows) else np.array([])
            if end_of_flag_indices.size == 0:
                logger.warning(f"No valid end for suspected Bullish Flag starting at index {i}. Skipping.")
                continue
            end_of_flag_index = i + 1 + end_of_flag_indices[0]

            if end_of_flag_index - i <= 15:
                logger.warning(f"Suspected Bullish Flag at index {i} is too short. Skipping.")
                continue

            # Find low of flag (lowest low between i and end_of_flag_index)
            low_of_flag = lows[i:end_of_flag_index + 1].min()
            low_of_flag_index = i + np.where(lows[i:end_of_flag_index + 1] == low_of_flag)[0][-1]

            # Find start of flag (first low lower than low_of_flag)
            start_of_flag_index = None
            for j in range(1, i):
                if highs[i-j] >= high_of_flag:
                    break
                elif lows[i-j] < low_of_flag:
                    start_of_flag_index = i-j
                    break
            
            if start_of_flag_index is None:
                logger.warning(f"No valid start point for suspected Bullish Flag starting at index {i}. Skipping.")
                continue
            
            # checking if is not a minor flag
            minor_indexes = np.where(highs[i+1:end_of_flag_index] > high_of_flag)[0] if i+1 < len(lows) else np.array([])
            if minor_indexes.size > 0:
                if minor_indexes[0] + i + 1 < low_of_flag_index:
                    logger.warning(f"Flag starting at index {i} is a minor flag. Skipping...")
                    continue
            
            # finding valid indexes for EL
            start_indices_EL = np.where((highs[i + 1:] > high_of_flag) & (lows[i + 1:] > high_of_flag))[0] if i+1 < len(lows) else np.array([])
            if start_indices_EL.size == 0:
                logger.warning(f"No valid start index for EL DP for suspected Bullish Flag starting at index {i}")
                start_index_EL = None
            else:
                used_flag_indices =  np.where((lows[end_of_flag_index:] < low_of_flag))[0]
                if used_flag_indices.size != 0 and used_flag_indices[0] + end_of_flag_index < start_indices_EL[0] + i + 1:
                    logger.warning(f"No valid start index in valid range for EL DP for suspected Bullish Flag starting at index {i}")
                    start_index_EL = None
                else:
                    start_index_EL = i + 1 + start_indices_EL[0]
            # Append valid flag
            flag = Flag(
                flag_id= self.dataset['time'][i],
                flag_type=("Bullish" if low_of_flag_index != i else "Undefined"),
                high=FlagPoint(price= high_of_flag,index= i),
                low=FlagPoint(price= low_of_flag, index= low_of_flag_index),
                data_in_flag= self.dataset.iloc[start_of_flag_index:end_of_flag_index + 1],
                start_index= start_of_flag_index,
                end_index= end_of_flag_index,
                start_FTC= end_of_flag_index,
                start_EL= start_index_EL
            )
            self.add_flag(flag)
    def detect_bearish_flags(self):
        """Detect Bearish Flags."""
        logger.info("Detecting Bearish Flags...")
        highs = self.dataset['high']
        lows = self.dataset['low']

        local_min_indices = np.where(self.dataset['is_local_min'].to_numpy())[0]
        for i in local_min_indices:
            low_of_flag = lows[i]

            # Find end of flag
            #end_of_flag_indices = np.where((lows[i + 1:] < low_of_flag) & (highs[i + 1:] < low_of_flag))[0] if i+1 < len(lows) else np.array([])
            end_of_flag_indices = np.where((lows[i + 1:] < low_of_flag))[0] if i+1 < len(lows) else np.array([])
            if end_of_flag_indices.size == 0:
                logger.warning(f"No valid end for suspected Bearish Flag starting at index {i}. Skipping.")
                continue
            end_of_flag_index = i + 1 + end_of_flag_indices[0]

            if end_of_flag_index - i <= 15:
                logger.warning(f"Suspected Bearish Flag at index {i} is too short. Skipping.")
                continue

            # Find high of flag (highest high between i and end_of_flag_index)
            high_of_flag = highs[i:end_of_flag_index + 1].max()
            high_of_flag_index = i + np.where(highs[i:end_of_flag_index + 1] == high_of_flag)[0][-1]

            # Find start of flag (first high lower than high_of_flag)
            start_of_flag_index = None
            for j in range(1, i):
                if lows[i-j] <= low_of_flag:
                    break
                elif highs[i-j] > high_of_flag:
                    start_of_flag_index = i-j
                    break
            
            if start_of_flag_index is None:
                logger.warning(f"No valid start point for suspected Bearish Flag starting at index {i}. Skipping.")
                continue

            # checking if is not a minor flag
            minor_indexes = np.where(lows[i+1:end_of_flag_index] < low_of_flag)[0] if i+1 < len(lows) else np.array([])
            if minor_indexes.size > 0:
                if minor_indexes[0] + i + 1 < high_of_flag_index:
                    logger.warning(f"Flag starting at index {i} is a minor flag so we are going to skip it")
                    continue  
            
            # finding valid indexes for EL
            start_indices_EL = np.where((lows[i + 1:] < low_of_flag) & (highs[i + 1:] < low_of_flag))[0] if i+1 < len(lows) else np.array([])
            if start_indices_EL.size == 0:
                logger.warning(f"No valid start index for EL DP for suspected Bearish Flag starting at index {i}")
                start_index_EL = None
            else:
                used_flag_indices =  np.where((highs[end_of_flag_index:] > high_of_flag))[0]
                if used_flag_indices.size != 0 and used_flag_indices[0] + end_of_flag_index < start_indices_EL[0] + i + 1:
                    logger.warning(f"No valid start index in valid range for EL DP for suspected Bearish Flag starting at index {i}")
                    start_index_EL = None
                else:
                    start_index_EL = i + 1 + start_indices_EL[0]

            # Append valid flag
            flag = Flag(
                flag_id= self.dataset['time'][i],
                flag_type= ("Bearish" if high_of_flag_index != i else "Undefined"),
                high=FlagPoint(price= high_of_flag, index= high_of_flag_index),
                low=FlagPoint(price= low_of_flag, index= i),
                data_in_flag= self.dataset.iloc[start_of_flag_index:end_of_flag_index + 1],
                start_index= start_of_flag_index,
                end_index= end_of_flag_index,
                start_FTC= end_of_flag_index,
                start_EL= start_index_EL
            )
            self.add_flag(flag)
            
    def run_detection(self):
        """Run flag detection for both Bullish and Bearish flags."""
        try:
            self.detect_local_extremes()  # Precompute local extremes
            self.detect_bullish_flags()
            self.detect_bearish_flags()
            logger.info("|||||||||||||| Detection complete ||||||||||||||||||")
        except Exception as e:
            logger.error(f"An error occurred during detection: {e}")
            raise

    def add_flag(self, flag: Flag):
        if flag.flag_id in self.flags:
            logger.warning(f"Flag {flag.flag_id} already exists. Updating the flag.")
        else:
            logger.info(f"Adding new flag: {flag}")
        self.flags[flag.flag_id] = flag  # Add or update the flag in the dictionary

    def remove_flag(self, flag_id):
        if flag_id not in self.flags:
            logger.error(f"Flag {flag_id} not found. Cannot delete.")
            return
        del self.flags[flag_id]  # Remove the flag from the dictionary
        logger.info(f"Flag {flag_id} removed.")

    def get_flag(self, flag_id):
        if flag_id in self.flags:
            logger.info(f"Retrieved flag {flag_id}: {self.flags[flag_id]}")
            return self.flags[flag_id]
        else:
            logger.warning(f"Flag {flag_id} not found.")
            return None

In [66]:
detector = FlagDetector(DataSet)
detector.run_detection()

In [67]:
FLAGS = pd.DataFrame(detector.flags.items(),columns= ["Flag Id", "Flag informations"])
FLAGS

,Flag Id,Flag informations
0,1722285900,"Flag is like ID: 1722285900, Type: Bullish, Hi..."
1,1722370500,"Flag is like ID: 1722370500, Type: Bullish, Hi..."
2,1722407400,"Flag is like ID: 1722407400, Type: Bullish, Hi..."
3,1722463200,"Flag is like ID: 1722463200, Type: Bullish, Hi..."
4,1722531600,"Flag is like ID: 1722531600, Type: Bullish, Hi..."
...,...,...
232,1734475500,"Flag is like ID: 1734475500, Type: Bearish, Hi..."
233,1734516900,"Flag is like ID: 1734516900, Type: Bearish, Hi..."
234,1734560100,"Flag is like ID: 1734560100, Type: Bearish, Hi..."
235,1734634800,"Flag is like ID: 1734634800, Type: Bearish, Hi..."


In [68]:
i = 0
print('flag type =')
print(FLAGS['Flag informations'][i].flag_type)

print('FTC =')
print(FLAGS['Flag informations'][i].FTC.DP)

print('EL =')
print(FLAGS['Flag informations'][i].EL.DP)

print('high=')
print(FLAGS['Flag informations'][i].high)

print('low=')
print(FLAGS['Flag informations'][i].low)

print('end of flag=')
print(FLAGS['Flag informations'][i].End_index)

print('EL start index=')
print(FLAGS['Flag informations'][i].EL.DP.start_index)

flag type =
Bullish
FTC =
DP_Parameteres(High=FlagPoint(price=1.082, index=72), Low=FlagPoint(price=1.08155, index=70), Status='Active', weight=1, start_index=97)
EL =
DP_Parameteres(High=FlagPoint(price=1.08238, index=33), Low=FlagPoint(price=1.08149, index=36), Status='Active', weight=1, start_index=102)
high=
FlagPoint(price=1.08259, index=43)
low=
FlagPoint(price=1.0814300000000001, index=74)
end of flag=
97
EL start index=
102


In [69]:
class Figure_Flag:
    def __init__(self, dataset: pd.DataFrame, FLAGS: pd.DataFrame):
        self.dataset = dataset
        self.FLAGS = FLAGS
        
    def basic_figure(self):
        # Extract high prices and indices for Bullish and Bearish flags
        High_figure = {
            "Bullish": [flag.high.price for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bullish'],
            "Bearish": [flag.high.price for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bearish']
        }

        Low_figure = {
            "Bullish": [flag.low.price for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bullish'],
            "Bearish": [flag.low.price for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bearish']
        }
        Time_figure = {
            "High_Bullish": [flag.high.index for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bullish'],
            "High_Bearish": [flag.high.index for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bearish'],
            "Low_Bullish": [flag.low.index for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bullish'],
            "Low_Bearish": [flag.low.index for flag in self.FLAGS['Flag informations'] if flag.flag_type == 'Bearish']
        }

        Important_DPs_figure = {
            "FTC_High": [flag.FTC.DP.High.price for flag in self.FLAGS['Flag informations']],
            "FTC_Lows": [flag.FTC.DP.Low.price for flag in self.FLAGS['Flag informations']],

            "EL_High": [flag.EL.DP.High.price for flag in self.FLAGS['Flag informations']],
            "EL_Lows": [flag.EL.DP.Low.price for flag in self.FLAGS['Flag informations']]
        }
        Time_DPs_figure = {
            "FTC_High": [flag.FTC.DP.High.index for flag in self.FLAGS['Flag informations']],
            "FTC_Lows": [flag.FTC.DP.Low.index for flag in self.FLAGS['Flag informations']],

            "EL_High": [flag.EL.DP.High.index for flag in self.FLAGS['Flag informations']],
            "EL_Lows": [flag.EL.DP.Low.index for flag in self.FLAGS['Flag informations']]
        }

        # Generate a list of indexes
        indexes = list(range(len(self.dataset['high'])))

        self.fig = go.Figure()

        # Add the first line (e.g., 'close' prices)
        self.fig.add_trace(go.Scatter(x=Time_figure['High_Bullish'], y=High_figure['Bullish'], mode='markers', name='High of Bullish Flags'))
        self.fig.add_trace(go.Scatter(x=Time_figure['High_Bearish'], y=High_figure['Bearish'], mode='markers', name='High of Bearish Flags'))


        self.fig.add_trace(go.Scatter(x=Time_figure['Low_Bullish'], y=Low_figure['Bullish'], mode='markers', name='Low of Bullish Flags'))
        self.fig.add_trace(go.Scatter(x=Time_figure['Low_Bearish'], y=Low_figure['Bearish'], mode='markers', name='Low of Bearish Flags'))

        self.fig.add_trace(go.Scatter(x=Time_DPs_figure['FTC_Lows'], y=Important_DPs_figure['FTC_Lows'], mode='markers', name='Low of FTC'))
        self.fig.add_trace(go.Scatter(x=Time_DPs_figure['FTC_High'], y=Important_DPs_figure['FTC_High'], mode='markers', name='High of FTC'))
        
        self.fig.add_trace(go.Scatter(x=Time_DPs_figure['EL_Lows'], y=Important_DPs_figure['EL_Lows'], mode='markers', name='Low of EL'))
        self.fig.add_trace(go.Scatter(x=Time_DPs_figure['EL_High'], y=Important_DPs_figure['EL_High'], mode='markers', name='High of EL'))
        # Add the third line (e.g., 'High' prices)
        self.fig.add_trace(go.Scatter(x=indexes, y=DataSet['high'], mode='lines', name='High Prices'))
        # Add the second line (e.g., 'Low' prices)
        self.fig.add_trace(go.Scatter(x=indexes, y=DataSet['low'], mode='lines', name='Low Prices'))

        # Customize layout if needed
        self.fig.update_layout(title="Highs and Lows on Close price",
                        xaxis_title="Time",
                        yaxis_title="Price")

        # Show the figure
        self.fig.show()
        

In [70]:
figure_of_Flag_15m = Figure_Flag(DataSet,FLAGS)
figure_of_Flag_15m.basic_figure()

In [71]:
def Reaction_to_DP(DP: DP_Parameteres, 
                   dataset: pd.DataFrame, 
                   direction: typing.Literal["Bullish", "Bearish","Undefined"],
                   start: int):
    reaction = None
    if direction == "Bullish" and DP.High.price is not None and DP.Low.price is not None and start is not None:
        # Find the initial index where 'low' crosses below DP.High.price
        initial_indexes = dataset.index[dataset["low"] <= DP.High.price]
        if initial_indexes.empty:
            return None
        
        initial_index = initial_indexes[0]

        # Find the close index where 'low' crosses below DP.Low.price
        close_indexes = dataset.index[initial_index + 1 - start:][dataset["low"][initial_index + 1-start:] <= DP.Low.price]
        close_index = close_indexes[0] if not close_indexes.empty else len(dataset) - 1

        # Calculate reaction
        TP = dataset["high"].iloc[initial_index - start:close_index - start].max()
        reaction = (TP - DP.High.price) / (DP.High.price - DP.Low.price)
        DP.Status = "Used"

        reaction *= DP.weight
        
    elif direction == "Bearish" and DP.High.price is not None and DP.Low.price is not None and start is not None: 
        # Find the initial index where 'high' crosses higher DP.Low.price
        initial_indexes = dataset.index[dataset["high"] >= DP.Low.price]
        if initial_indexes.empty:
            return None
        
        initial_index = initial_indexes[0]

        # Find the close index where 'high' crosses higher DP.High.price
        close_indexes = dataset.index[initial_index + 1 -start:][dataset["high"][initial_index + 1 -start:] >= DP.High.price]
        close_index = close_indexes[0] if not close_indexes.empty else len(dataset) - 1

        # Calculate reaction
        TP = dataset["low"].iloc[initial_index -start:close_index - start].min()
        reaction = (-TP + DP.Low.price) / (DP.High.price - DP.Low.price)
        DP.Status = "Used"

        reaction *= DP.weight
    return reaction

In [72]:
def Reaction_detector(flags: pd.DataFrame, dataset: pd.DataFrame):
    Reaction_to_FTC = []
    Reaction_to_EL = []
    for flag in flags['Flag informations']:
        Reaction_to_EL.append(Reaction_to_DP(flag.EL.DP, dataset.iloc[flag.EL.DP.start_index:], flag.flag_type, flag.EL.DP.start_index))
        Reaction_to_FTC.append(Reaction_to_DP(flag.FTC.DP, dataset.iloc[flag.FTC.DP.start_index:], flag.flag_type, flag.FTC.DP.start_index)) 
    
    flags["Reaction to FTC"] = Reaction_to_FTC
    flags["Reaction to EL"] = Reaction_to_EL

In [73]:
Reaction_detector(FLAGS, DataSet)

In [74]:
FLAGS

,Flag Id,Flag informations,Reaction to FTC,Reaction to EL
0,1722285900,"Flag is like ID: 1722285900, Type: Bullish, Hi...",3.488889,0.438202
1,1722370500,"Flag is like ID: 1722370500, Type: Bullish, Hi...",3.838710,1.985507
2,1722407400,"Flag is like ID: 1722407400, Type: Bullish, Hi...",0.396825,2.033333
3,1722463200,"Flag is like ID: 1722463200, Type: Bullish, Hi...",0.317073,NaN
4,1722531600,"Flag is like ID: 1722531600, Type: Bullish, Hi...",0.763359,0.476415
...,...,...,...,...
232,1734475500,"Flag is like ID: 1734475500, Type: Bearish, Hi...",NaN,NaN
233,1734516900,"Flag is like ID: 1734516900, Type: Bearish, Hi...",NaN,NaN
234,1734560100,"Flag is like ID: 1734560100, Type: Bearish, Hi...",NaN,NaN
235,1734634800,"Flag is like ID: 1734634800, Type: Bearish, Hi...",0.578199,NaN


In [75]:
indexes = list(range(len(FLAGS['Reaction to FTC'])))
fig = go.Figure()
fig.add_trace(go.Scatter(x=indexes, y=FLAGS["Reaction to FTC"], mode='markers', name='Reaction to FTC'))
fig.add_trace(go.Scatter(x=indexes, y=FLAGS["Reaction to EL"], mode='markers', name='Reaction to EL'))

In [76]:
# Function to calculate profits and winrates for a given reaction column
def calculate_metrics(FLAGS, column_name, RR_values, commision, R):
    RESULT = []
    Winrates = []

    for RR in RR_values:
        result = 0
        Total_SLs = 0
        flag_index = -1
        trades =0

        # Iterate through reactions
        for value in FLAGS[column_name]:
            flag_index += 1
            if pd.isna(value):  # Skip NaN values
                continue

            trades += 1
            weight = FLAGS['Flag informations'][flag_index].weight
            if value <= RR:
                Total_SLs += 1
                result -= R*weight
            elif value > RR:
                result += R*RR*weight

            result -= commision

    
        winrate = 1 - (Total_SLs / trades)
        RESULT.append(result)
        Winrates.append(winrate * 100)
        """
    # Plot Results
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.array(Winrates), y=np.array(RESULT), mode='lines', name=column_name))
    #fig.add_trace(go.Scatter(x=np.array(Winrates_bullish), y=np.array(RESULT_bullish), mode='lines', name=column_name + "bullish"))
    #fig.add_trace(go.Scatter(x=np.array(Winrates_bearish), y=np.array(RESULT_bearish), mode='lines', name=column_name + "bearish"))
    fig.update_layout(title=f"RESAULT {column_name}",
                    xaxis_title="Winrate (%)",
                    yaxis_title=f"RESAULT (R={R}%)")
    fig.show()
    # Plot Results
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=RR_values, y=np.array(RESULT) * np.array(Winrates) / 100, mode='lines', name=column_name))
    #fig.add_trace(go.Scatter(x=RR_values, y=np.array(RESULT_bullish) * np.array(Winrates_bullish) / 100, mode='lines', name=column_name+ "bullish"))
    #fig.add_trace(go.Scatter(x=RR_values, y=np.array(RESULT_bearish) * np.array(Winrates_bearish) / 100, mode='lines', name=column_name+ "bearish"))
    fig.update_layout(title=f"WinRate * RESULT vs RR in {column_name}",
                    xaxis_title="RR",
                    yaxis_title="WinRate * RESULT")
    fig.show()
    """
    # Plot Results
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=RR_values, y=100*np.array(RESULT)/account_info.balance, mode='lines', name="Profit"))
    fig.add_trace(go.Scatter(x=RR_values, y=np.array(Winrates), mode='lines', name="Winrate"))
    #fig.add_trace(go.Scatter(x=RR_values, y=np.array(RESULT_bullish), mode='lines', name=column_name+ "bullish"))
    #fig.add_trace(go.Scatter(x=RR_values, y=np.array(RESULT_bearish), mode='lines', name=column_name+ "bearish"))
    fig.update_layout(title=column_name,
                    xaxis_title="RR",
                    yaxis_title="Percentage")
    fig.show()

    # Identify valid RRs
    valid_RRs = RR_values[(np.array(RESULT) == np.array(RESULT).max())]
    print(f"Valid RRs based on {column_name}: {valid_RRs}")

    print (f"total number of trades:{trades}")
    #return np.array(RESULT), np.array(Winrates)
    return

# Constants
R = (config["risk_management"]["R"] /100 ) * account_info.balance
commision = config["account_info"]["commision"]
RR_values = np.arange(0.1, 10, 0.01)

# Calculate metrics
#Profits_EL, Winrates_EL = calculate_metrics(FLAGS, 'Reaction to EL', RR_values, commision, R)
#Profits_FTC, Winrates_FTC = calculate_metrics(FLAGS, 'Reaction to FTC', RR_values, commision, R)
calculate_metrics(FLAGS, 'Reaction to EL', RR_values, commision, R)
calculate_metrics(FLAGS, 'Reaction to FTC', RR_values, commision, R)

Valid RRs based on Reaction to EL: [1.39]
total number of trades:128


Valid RRs based on Reaction to FTC: [2.03]
total number of trades:193


In [64]:
unvalid = {
    "ftc" :  0,
    "el" : 0,
    "undefined" : 0    
}
unvalid_EL_bearish=0
for flag in FLAGS['Flag informations']:
    if flag.FTC.DP.Status == "Not Valid":
        unvalid["ftc"] +=1
    if flag.EL.DP.Status == "Not Valid":
        unvalid['el'] +=1
    if  flag.flag_type == "Undefined":
        unvalid["undefined"] += 1

print(unvalid)

{'ftc': 23, 'el': 67, 'undefined': 4}
